In [1]:
import pandas as pd
import numpy as np
import networkx as nx 
from sklearn.metrics import mean_squared_error 
from numpy.linalg import norm

In [2]:
# # from colab run this 
# !git clone https://github.com/gru13/lncRNA_Disease_association.git
# df_lnc_die = pd.read_csv(r"./lncRNA_Disease_association/datasets/lncrnaDiesease3/website_alldata_p1.csv")
# df_lnc_mir = pd.read_csv(r"./lncRNA_Disease_association/datasets/LncRNASNP/Homo_sapiens_lncRNA_miRNA_interactions.csv")
# df_mir_die = pd.read_csv(r"./lncRNA_Disease_association/datasets/rnaDisease/RNADisease_RNA-disease_miRNA_predict.csv")

In [3]:
# from local system

df_lnc_die = pd.read_csv(r"./../datasets/lncrnaDiesease3/website_alldata_p1.csv")
df_lnc_mir = pd.read_csv(r"./../datasets/LncRNASNP/Homo_sapiens_lncRNA_miRNA_interactions.csv")
df_mir_die = pd.read_csv(r"./../datasets/rnaDisease/RNADisease_RNA-disease_miRNA_predict.csv")


In [4]:

df_lnc_die.head(),df_lnc_mir.head(),df_mir_die.head()

(        lncRNA                    Disease  PubMed ID
 0  ARHGAP5-AS1  Carcinoma, Hepatocellular   36354136
 1       HOTTIP               Osteosarcoma   33475442
 2       HOTTIP                     Glioma   35402278
 3       HOTTIP             Retinoblastoma   33784880
 4       HOTTIP          stomach carcinoma   32633359,
             lncRNA           miRNA
 0  NONHSAT000002.2  hsa-miR-15a-5p
 1  NONHSAT000002.2  hsa-miR-15a-3p
 2  NONHSAT000002.2   hsa-miR-16-5p
 3  NONHSAT000002.2   hsa-miR-21-3p
 4  NONHSAT000002.2   hsa-miR-28-5p,
           miRNA                    Disease
 0  hsa-let-7a-1  Abdominal aortic aneurysm
 1  hsa-let-7a-2  Abdominal aortic aneurysm
 2  hsa-let-7a-3  Abdominal aortic aneurysm
 3    hsa-let-7c  Abdominal aortic aneurysm
 4    hsa-let-7d  Abdominal aortic aneurysm)

In [5]:
dataset1 = pd.merge(df_lnc_mir,df_lnc_die,on=['lncRNA'],how='inner')
dataset2 = pd.merge(df_lnc_mir,df_mir_die,on=['miRNA'],how='inner')

In [6]:
# dataset1

In [7]:
dataset2

,lncRNA,miRNA,Disease
0,NONHSAT000002.2,hsa-miR-15a-5p,Abortion habitual
1,NONHSAT000002.2,hsa-miR-15a-5p,Acoustic neuroma
2,NONHSAT000002.2,hsa-miR-15a-5p,Acquired immunodeficiency syndrome
3,NONHSAT000002.2,hsa-miR-15a-5p,Acute coronary syndrome
4,NONHSAT000002.2,hsa-miR-15a-5p,Acute kidney failure
...,...,...,...
9376652,NONHSAT017163.2,hsa-miR-523-3p,Intellectual disability
9376653,NONHSAT017163.2,hsa-miR-523-3p,Leukemia
9376654,NONHSAT017163.2,hsa-miR-523-3p,Neurodegenerative diseases
9376655,NONHSAT017163.2,hsa-miR-523-3p,Skin disease


In [8]:
df = dataset2.copy()
# df

In [9]:
df = dataset2.copy()

In [10]:
# # only run this for first time 
# disease = df['Disease'].unique()
# lncRNA = df['lncRNA'].unique()
# miRNA = df['miRNA'].unique()

# np.savetxt("./disease.txt",disease,delimiter=',',fmt='%s')
# np.savetxt("./lncRNA.txt",lncRNA,delimiter=',',fmt='%s')
# np.savetxt("./miRNA.txt",miRNA,delimiter=',',fmt='%s')

In [11]:
disease = [a[:-1] for a in open("./disease.txt").readlines()]
lncRNA = [a[:-1] for a in open("./lncRNA.txt").readlines()]
miRNA = [a[:-1] for a in open("./miRNA.txt").readlines()]

In [12]:
len(lncRNA),len(miRNA),len(disease)

(7554, 239, 514)

In [13]:
lncRNA_index = {a:b for (a,b) in zip(lncRNA,range(len(lncRNA)))}
disease_index = {a:b for (a,b) in zip(disease,range(len(disease)))}
miRNA_index = {a:b for (a,b) in zip(miRNA,range(len(miRNA)))}

In [14]:
# len(df['lncRNA'].unique()),len(df['miRNA'].unique()),len(df['Disease'].unique()),

CReation of the graph that contains the totaly connected with miRNA, lncRNA, Disease

In [15]:
LM_Graph = nx.Graph()
MD_Graph = nx.Graph()
LD_Graph = nx.Graph()

LM_Graph.add_nodes_from(lncRNA)
LM_Graph.add_nodes_from(miRNA)

LD_Graph.add_nodes_from(lncRNA)
LD_Graph.add_nodes_from(disease)

MD_Graph.add_nodes_from(miRNA)
MD_Graph.add_nodes_from(disease)

In [16]:
for a in df.values:
    LM_Graph.add_edge(a[0],a[1])
    MD_Graph.add_edge(a[1],a[2])
    LD_Graph.add_edge(a[0],a[2])

CReation of the Lncrna-disease , Lncrna-miRNA, miRNA-Disease assoction matrix (LM,LD,MD)

In [17]:
LM_index = {a:b for (a,b) in zip(LM_Graph.nodes(),range(len(LM_Graph.nodes())))}
MD_index = {a:b for (a,b) in zip(MD_Graph.nodes(),range(len(MD_Graph.nodes())))}
LD_index = {a:b for (a,b) in zip(LD_Graph.nodes(),range(len(LD_Graph.nodes())))}

In [18]:
LM_M_A = nx.adjacency_matrix(LM_Graph).todense()
MD_M_A = nx.adjacency_matrix(MD_Graph).todense()
LD_M_A = nx.adjacency_matrix(LD_Graph).todense()

In [19]:
LM_M = np.zeros(shape=(len(lncRNA),len(miRNA)),dtype=int)
MD_M = np.zeros(shape=(len(miRNA), len(disease)),dtype=int)
LD_M = np.zeros(shape=(len(lncRNA), len(disease)),dtype=int)

In [20]:
for a in range(len(lncRNA)):
    for b in range(len(disease)):
        LD_M[a][b] = LD_M_A[LD_index[lncRNA[a]]][LD_index[disease[b]]]
LD_M

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
for a in range(len(lncRNA)):
    for b in range(len(miRNA)):
        LM_M[a][b] = LM_M_A[LM_index[lncRNA[a]]][LM_index[miRNA[b]]]
LM_M

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
for a in range(len(miRNA)):
    for b in range(len(disease)):
        MD_M[a][b] = MD_M_A[MD_index[miRNA[a]]][MD_index[disease[b]]]
MD_M

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
LD = LM_M.dot(MD_M)
LD

array([[5, 6, 6, ..., 0, 0, 0],
       [3, 5, 5, ..., 0, 0, 0],
       [7, 9, 7, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
# # run this if CD.csv is not found in the current folder(runs for seconds)
# CD = np.zeros(shape=(len(disease),len(disease)))
# for i in range(len(disease)):
#     for j in range(len(disease)):
#         CD[i][j] = (MD_M[:,i].dot(MD_M[:,j]))/(norm(MD_M[:,i])*norm(MD_M[:,j]))
# np.save("CD",CD)

In [25]:
CD = np.load("CD.npy")

In [26]:
# # run this if CL.csv not found in the current folder(run for 8 min )
# CL = np.zeros(shape=(len(lncRNA),len(lncRNA)))
# for i in range(len(lncRNA)):
#     for j in range(len(lncRNA)):
#         CL[i][j] = (LM_M[i,:].dot(LM_M[j,:]))/(norm(LM_M[i,:])*norm(LM_M[j,:]))
# np.save("CL",CL)

In [27]:
CL = np.load("CL.npy")

In [28]:
JD = np.zeros(shape=CD.shape)
for i in range(MD_M.shape[-1]):
    for j in range(MD_M.shape[-1]):
        JD[i][j] =(np.bitwise_and(MD_M[:,i],MD_M[:,j]).sum())/(np.bitwise_or(MD_M[:,i],MD_M[:,j]).sum())
        

In [29]:
JL = np.zeros(shape=CL.shape)
for i in range(LM_M.shape[-1]):
    for j in range(LM_M.shape[-1]):
        JL[i][j] =(np.bitwise_and(LM_M[:,i],LM_M[:,j]).sum())/(np.bitwise_or(LM_M[:,i],LM_M[:,j]).sum())
JL      

array([[1.        , 0.28849315, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28849315, 1.        , 0.28849315, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.28849315, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [30]:
IDS = np.zeros(shape=CD.shape)
for i in range(IDS.shape[0]):
    for j in range(IDS.shape[-1]):
        if CD[i][j] == 0:
            IDS[i][j] = JD[i][j]
        else:
            IDS[i][j] = (CD[i][j]+JD[i][j])/2
IDS

array([[1.        , 0.73534833, 0.62628227, ..., 0.        , 0.        ,
        0.        ],
       [0.73534833, 1.        , 0.79003636, ..., 0.19620955, 0.19620955,
        0.        ],
       [0.62628227, 0.79003636, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.19620955, 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.19620955, 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [31]:
# ILS = np.zeros(shape=CL.shape)
# for i in range(ILS.shape[0]):
#     for j in range(ILS.shape[-1]):
#         if CL[i][j] == 0:
#             ILS[i][j] = JL[i][j]
#         else:
#             ILS[i][j] = (CL[i][j]+JL[i][j])/2
# np.save("ILS",ILS)
# ILS

In [32]:
ILS = np.load("ILS.npy")

Dormant Factor model

In [33]:
# # Run this only if u don't have phi.npy(this runs for 5-8 min)
# def Dormant_factor_model(ALM, AMD, k, alpha=2*10**-6, lmbda=4*10**-5, max_iter=1000):
#     """
#     Dormant factor model for calculating lncRNA-disease associations.

#     Args:
#         ALM (numpy.ndarray): Adjacency matrix of lncRNA-miRNA associations (m x n).
#         AMD (numpy.ndarray): Adjacency matrix of miRNA-disease associations (n x e).
#         k (int): Number of Dormant factors.
#         alpha (float): Learning rate for gradient descent.
#         lmbda (float): Regularization parameter.
#         max_iter (int): Maximum number of iterations for gradient descent.

#     Returns:a
#         X (numpy.ndarray): lncRNA feature matrix (m x k).
#         Y (numpy.ndarray): Disease feature matrix (e x k).
#         psi (numpy.ndarray): lncRNA-disease association score matrix (m x e).
#     """


#     # Calculate the preliminary lncRNA-disease association matrix
#     ALD = ALM @ AMD

#     m = ALD.shape[0]
#     e = ALD.shape[1]

#     # Initialize feature matrices randomly
#     X = np.random.rand(m, k)
#     Y = np.random.rand(e, k)

#     for _ in range(max_iter):
#         # Calculate the lncRNA-disease association score matrix
#         psi = X @ Y.T   

#         # Calculate the loss function and gradients
#         # loss = np.sum((psi - ALD) ** 2) + lmbda * (np.sum(norm(X)) + np.sum(norm(Y)))
#         grad_X = 2 * (psi - ALD) @ Y +  lmbda * X
#         grad_Y = 2 * (psi - ALD).T @ X + lmbda * Y

#         # Update feature matrices using gradient descent
#         X -= alpha * grad_X
#         Y -= alpha * grad_Y

#     return X, Y, psi
# X,Y,phi = Dormant_factor_model(LM_M,MD_M,213,max_iter=3500,alpha=2*10**-6,lmbda=4*10**-6)
# mean_squared_error(phi,LM_M@MD_M),X,Y,phi
# np.save("phi",phi)
# #mse -> .16 

In [34]:
phi = np.load("phi.npy")

In [35]:
(LM_M@MD_M).max(),(LM_M@MD_M).min()

(201, 0)

In [36]:
phi.max(), phi.min()

(201.87444701561898, -1.8466962382494252)

In [37]:
# LP = np.zeros(shape=LD_M.shape)
# for i in range(LP.shape[0]):
#     for j in range(len(lncRNA)):
#         LP[i,:] += ILS[i,j]*phi[j,:] 
#     LP[i,:] /= norm(ILS[i:]) 
# np.save("LP",LP)
# LP

In [38]:
LP = np.load("LP.npy")

In [39]:
DP = np.zeros(shape=LD_M.shape)
for j in range(DP.shape[1]):
    for i in range(len(disease)):
        DP[:,j] += IDS[i,j]*phi[:,i]
    DP[:,j] /= norm(IDS[:,j])
DP

array([[ 1.03356487e+02,  1.05355505e+02,  1.04801229e+02, ...,
         4.34408789e+01,  4.34408789e+01,  2.70538526e+01],
       [ 7.21664586e+01,  7.40069411e+01,  7.39931303e+01, ...,
         3.05678821e+01,  3.05678821e+01,  1.47234207e+01],
       [ 1.27244263e+02,  1.29103802e+02,  1.26622868e+02, ...,
         5.34589406e+01,  5.34589406e+01,  1.62884687e+01],
       ...,
       [ 4.05422049e-01,  3.85547493e-01,  5.79096074e-01, ...,
        -5.10439681e-03, -5.10439681e-03,  3.51478660e-01],
       [ 4.38426165e-01,  4.02847840e-01,  4.81957536e-01, ...,
         7.63602076e-03,  7.63602076e-03,  5.45215725e-01],
       [ 3.07047778e-01,  1.61024016e-01,  2.40520009e-01, ...,
         4.19304785e-01,  4.19304785e-01,  2.57217981e-01]])

In [40]:
LP.shape,DP.shape,ILS.shape,IDS.shape

((7554, 514), (7554, 514), (7554, 7554), (514, 514))

In [41]:
LP.max(),DP.max(),ILS.max(),IDS.max()

(3774.748205640315, 438.7376207688449, 1.0, 1.0)

In [42]:
LFMP = np.zeros(shape=LD_M.shape)
w = 0.50
for i in range(LFMP.shape[0]):
    for j in range(LFMP.shape[1]):
        LFMP[i][j] = (w*LP[i][j] + (1-w)*DP[i][j])/(norm(ILS[i,:])+norm(IDS[:,j]))
LFMP

array([[1.59113727, 1.63968119, 1.71031294, ..., 0.907568  , 0.90756834,
        0.62106861],
       [1.28545532, 1.33549406, 1.4112091 , ..., 0.78428833, 0.78428887,
        0.42512103],
       [2.25192288, 2.30701784, 2.40251561, ..., 1.39618111, 1.39618196,
        0.48034345],
       ...,
       [1.84334398, 2.16379285, 1.96668324, ..., 0.17352366, 0.17346477,
        0.25633788],
       [2.95811104, 3.42715179, 3.15170635, ..., 0.23631644, 0.23623733,
        0.34261506],
       [4.69487994, 5.43083264, 5.03568208, ..., 0.41555581, 0.41536897,
        0.51386781]])

In [43]:
LFMP.max(),LFMP.min(),LFMP.mean(),LFMP.std()

(329.6302410451272,
 -0.09867926740107609,
 0.9944589650187683,
 1.114331934857895)

In [44]:
LFMP.shape

(7554, 514)

In [45]:
d_len = len(disease)
l = np.array([[a]*d_len for a in lncRNA]).flatten()
l.size

3882756

In [52]:
d = np.array(disease*len(lncRNA))
d.size

3882756

In [66]:
stat_d = pd.DataFrame()
stat_d['lncRNA'] = l.copy()
stat_d['diease'] = d.copy()
stat_d['Algorithmic Value'] = LFMP.flatten().copy()
stat_d['Algorithmic Value'] = stat_d['Algorithmic Value']

In [61]:
stat_d.to_csv("Latent_output.csv")

In [67]:
stat_d.to_feather("./Latent_output.feather")

In [68]:

k = pd.read_feather("./Latent_output.feather")
k

,lncRNA,diease,Algorithmic Value
0,NONHSAT000002.2,Abortion habitual,1.591137
1,NONHSAT000002.2,Acoustic neuroma,1.639681
2,NONHSAT000002.2,Acquired immunodeficiency syndrome,1.710313
3,NONHSAT000002.2,Acute coronary syndrome,1.642922
4,NONHSAT000002.2,Acute kidney failure,1.639706
...,...,...,...
3882751,NONHSAT007829.2,Childhood medulloblastoma,0.415320
3882752,NONHSAT007829.2,Differentiating neuroblastoma,0.415306
3882753,NONHSAT007829.2,Ganglioneuroblastoma,0.415556
3882754,NONHSAT007829.2,Ganglioneuroma,0.415369
